In [1]:
#!/usr/bin/python
'''Data Source:
http://guides.lib.purdue.edu/c.php?g=353290&p=2378621#Mississippi
'''
import os
from  osgeo import ogr, osr

def convert_cordinates(srcshapefile,destshapefile,target_epsg):
    srcshpfile=srcshapefile
    outputShapefile=destshapefile
    driver = ogr.GetDriverByName('ESRI Shapefile')
    shp = driver.Open(srcshpfile)
    layer = shp.GetLayer()
    sourceSpatialRef = layer.GetSpatialRef()
    outprjrefwm = osr.SpatialReference()
    if sourceSpatialRef.IsProjected:
        datum=sourceSpatialRef.GetAttrValue('GEOGCS|DATUM')
        if 'North_American_Datum_1983' in datum:
            outprjrefwm.ImportFromEPSG(target_epsg)
    transform= osr.CoordinateTransformation(sourceSpatialRef,outprjrefwm)
    #saving layer
    if os.path.exists(outputShapefile):
        driver.DeleteDataSource(outputShapefile)
    outDataSet = driver.CreateDataSource(outputShapefile)
    outLayer = outDataSet.CreateLayer("prjstates",outprjrefwm,geom_type=ogr.wkbMultiPolygon)
    # add fields
    inLayerDefn = layer.GetLayerDefn()
    for i in range(0, inLayerDefn.GetFieldCount()):
        fieldDefn = inLayerDefn.GetFieldDefn(i)
        outLayer.CreateField(fieldDefn)
    outLayerDefn = outLayer.GetLayerDefn()

    #read the features one by one.
    inFeature = layer.GetNextFeature()
    while inFeature:
        inputGeom = inFeature.GetGeometryRef()
        # get the cover attribute for the input feature
        inputGeom.TransformTo(outprjrefwm) #to WGS84/Pseudo Mercator
        outFeature = ogr.Feature(outLayerDefn)
        outFeature.SetGeometry(inputGeom)
        for i in range(0, outLayerDefn.GetFieldCount()):
            outFeature.SetField(outLayerDefn.GetFieldDefn(i).GetNameRef(), inFeature.GetField(i))
        outLayer.CreateFeature(outFeature)
        inFeature = None
        inFeature = layer.GetNextFeature()
    shp = None
    outDataSet = None
    print("completed cordinate transform.")

#%matplotlib inline  
# Import the necessary modules
epsgdic = {'nad83':4269,'wgs84':4326,'pseudoutm':3857,'worldmercater':3395}
home =r'/home/naresh-1/workspace/machinelrn/data/gis/'
srcshpfile = home + r'out/usa.shp'
outputShapefile = home+ r'out/prj_usa.shp'
#convert_cordinates(srcshpfile,outputShapefile,epsgdic['worldmercater'])

In [6]:
import ntpath
ntpath.basename("a/b/c.txt").split(".")[0]


'c'